# Integrating Preprocessing with Python [WIP]

In this example, we demonstrate how to use python to preprocess data.

## Prerequisites

We are going to find section properties from AISC table, which is available online.

In [ ]:
import pandas

section_table = pandas.read_excel(
    'https://www.aisc.org/globalassets/aisc/manual/v15.0-shapes-database/aisc-shapes-database-v15.0.xlsx', sheet_name=1,
    usecols='A:CF')
print(section_table.head())

We are going to extract section properties such as area and moment of inertia from this table using section designations. It is possible to define such a function now.

In [ ]:
def from_table(designation: str):
    index = section_table.index[section_table['AISC_Manual_Label'] == designation].tolist()
    assert len(index) == 1
    a = section_table.at[index[0], 'A']
    sx = section_table.at[index[0], 'Sx']
    ix = section_table.at[index[0], 'Ix']
    return a, sx, ix

## Geometry of the Frame Structure

For simplicity, we assume the frame structure has the same column/beam section for all the columns/beams on the same floor. Under such a condition, two lists of section designations can be provided so that elements can be created. Similarly, geometry information such as floor height, bay span, as well as floor mass, can be provided in the same way.

For example, we can define several lists as follows.

In [ ]:
girder = ['W21X68', 'W21X68', 'W21X68']  # floor 1 2 3
column = ['W14X193', 'W14X159', 'W14X159']  # column 1 2 3
mass = [30., 30., 30.]  # floor 1 2 3
span = [240., 240.]  # bay span 1 2
height = [120., 120., 120.]  # floor height 1 2 3

### Generate Node Grid

Given that span and height are given, it is possible to calculate the absolute position of the nodes.

In [ ]:
import numpy as np

x_coor = np.cumsum(span)
y_coor = np.cumsum(height)
x_coor = np.insert(x_coor, 0, 0.)
y_coor = np.insert(y_coor, 0, 0.)
print(x_coor, y_coor)

In [ ]:
node_grid = np.zeros((len(y_coor), len(x_coor))).astype(int)

node_tag = 1
with open('node.sp', 'w') as f:
    for i in range(len(x_coor)):
        for j in range(len(y_coor)):
            f.write(f'node {node_tag} {x_coor[i]:.2f} {y_coor[j]:.2f}\n')
            node_grid[j, i] = node_tag
            node_tag += 1

print(node_grid)

The `node_grid` can be used to generate elements.

### Generate Beam Elements

Starting with the second row, beam elements can be generated by looping over each row.

In [ ]:
element_tag = 1
material_tag = 1
with open('beam.sp', 'w') as f:
    for i in range(1, len(y_coor)):
        a, sx, ix = from_table(girder[i - 1])
        for j in range(len(x_coor) - 1):
            node_i = node_grid[i, j]
            node_j = node_grid[i, j + 1]
            f.write(f'element EB21 {element_tag} {node_i} {node_j} {a:.3f} {ix:.3f} {material_tag}\n')
            element_tag += 1

### Generate Column Elements

Similarly, column elements can be generated by looping over each column.

In [ ]:
with open('column.sp', 'w') as f:
    for i in range(len(y_coor) - 1):
        a, sx, ix = from_table(column[i])
        for j in range(len(x_coor)):
            node_i = node_grid[i, j]
            node_j = node_grid[i + 1, j]
            f.write(f'element EB21 {element_tag} {node_i} {node_j} {a:.3f} {ix:.3f} {material_tag}\n')
            element_tag += 1